# ダミーデータの参考情報

In [4]:
import random
from pyspark.sql.types import *
from pyspark.sql.functions import explode
from pyspark.sql.functions import udf
from pyspark.sql.functions import array
import numpy as np
import datetime as dt
import scipy.stats as stats

元データの読み込み

In [2]:
output_base_url = os.environ['OUTPUT_URL']
output_url = output_base_url + 'el_aircon'

In [3]:
written_df = spark.read.format('delta').load(output_url)
written_df

21/10/16 09:41:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[key_str: string, value_str: string, key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [4]:
written_df.show()

+------------+---------+--------------------+----------------+---------+---------+------+--------------------+-------------+
|     key_str|value_str|                 key|           value|    topic|partition|offset|           timestamp|timestampType|
+------------+---------+--------------------+----------------+---------+---------+------+--------------------+-------------+
|0472#0x0,0x1|    49,21|[30 34 37 32 23 3...|[34 39 2C 32 31]|el_aircon|        0|     0|2021-10-16 01:57:...|            0|
|0472#0x0,0x1|    49,21|[30 34 37 32 23 3...|[34 39 2C 32 31]|el_aircon|        0|     1|2021-10-16 01:57:...|            0|
|0472#0x0,0x1|    49,21|[30 34 37 32 23 3...|[34 39 2C 32 31]|el_aircon|        0|     2|2021-10-16 01:57:...|            0|
+------------+---------+--------------------+----------------+---------+---------+------+--------------------+-------------+



今回使用するのは、上記のうち `key_str` 、 `value_str` 、 `timestamp` あたり。他の値はひとまず不要。
そこでそのカラムを持ったダミーデータを作成する。

# ダミーデータ生成

In [43]:
#output_base_url = os.environ['OUTPUT_URL']
#output_url = output_base_url + 'sensor_data_01' # sensor_data_01 or sensor_data_02
output_url = 'data/sensor_data_02'              # sensor_data_01 or sensor_data_02
id_min = 1000                                      # sensor_data_01-> 0, sensor_data_02 -> 1000とした
id_max = 1999                                    # sensor_data_01 -> 999, sensor_data_02 -> 1999とした
num_id = 50

まずベースとなるIDの元の乱数を任意個生成し、それを使ってDataFrameを作成する。

In [44]:
def rand_ints_nodup(a, b, k):
  ns = []
  while len(ns) < k:
    n = random.randint(a, b)
    if not n in ns:
      ns.append(n)
  return ns

In [45]:
ids = rand_ints_nodup(id_min, id_max, num_id)
id_strs = list(map(lambda x: str(x).zfill(4), ids))

In [46]:
df1 = spark.createDataFrame(id_strs, StringType())

続いて、必要なカラムを作成するためのUDFを定義する。

In [47]:
# 乱数から作った値にECHONET LiteでのIDを付与する
def gen_key_str(id):
    return id + "#" + "0x0,0x1"

gen_key_str_udf = udf(gen_key_str)

In [48]:
# ある日付の1時間ごとの電源状態（On/Off）と温度設定値の乱数を生成し、配列として返す
def gen_value_str():
    records = []
    num_records = 24 # 1時間ごとに24時間分
    
    # 温度設定の乱数生成用。ここでは上限・下限のある正規分布を用いる
    lower, upper= 20, 30
    mu, sigma= 27, 3
    X = stats.truncnorm((lower -mu) /sigma, (upper -mu) /sigma, loc=mu, scale=sigma)
    temps = X.rvs(num_records)
        
    timestamp = dt.datetime(2021, 1, 6)
    one_hour = dt.timedelta(hours=1)
    
    for i in range(0, num_records):
        state = 49 if random.randint(0, 2) == 0 else 48  # On（48）の方がOff（49）より2倍程度確率が高くなるように調整。
        temp = temps[i]
        records.append(("%d,%d" % (state, temp), timestamp))
        timestamp = timestamp + one_hour
        
    return records

gen_value_str_udf = udf(gen_value_str, ArrayType(StructType([StructField('value_str', StringType(), False), StructField('timestamp', TimestampType(), False)])))

キー（レコードを送ってきた主のID）、値（電源状態、温度設定値）、タイムスタンプ（ある日付の1時間ごと）を含んだデータを生成する。
なお、この時点では値とタイムスタンプは構造化され、さらに行内で配列化されている。

In [49]:
df2 = df1.select(gen_key_str_udf('value').alias('key_str'), 
                 gen_value_str_udf().alias('value_str'))

In [50]:
df2.schema

StructType(List(StructField(key_str,StringType,true),StructField(value_str,ArrayType(StructType(List(StructField(value_str,StringType,false),StructField(timestamp,TimestampType,false))),true),true)))

配列となっている値とタイムスタンプのペアを行に展開する。

In [51]:
df3 = df2.select('key_str', explode('value_str').alias('val_time'))

構造化された値とタイムスタンプを異なる列に展開する。

In [52]:
df4 = df3.select('key_str', 'val_time.*')

結果は以下の通り。

In [53]:
df4.show(30)

+------------+---------+-------------------+
|     key_str|value_str|          timestamp|
+------------+---------+-------------------+
|1325#0x0,0x1|    48,28|2021-01-06 00:00:00|
|1325#0x0,0x1|    49,21|2021-01-06 01:00:00|
|1325#0x0,0x1|    49,25|2021-01-06 02:00:00|
|1325#0x0,0x1|    48,26|2021-01-06 03:00:00|
|1325#0x0,0x1|    48,25|2021-01-06 04:00:00|
|1325#0x0,0x1|    49,20|2021-01-06 05:00:00|
|1325#0x0,0x1|    48,26|2021-01-06 06:00:00|
|1325#0x0,0x1|    48,26|2021-01-06 07:00:00|
|1325#0x0,0x1|    48,21|2021-01-06 08:00:00|
|1325#0x0,0x1|    48,29|2021-01-06 09:00:00|
|1325#0x0,0x1|    48,26|2021-01-06 10:00:00|
|1325#0x0,0x1|    48,26|2021-01-06 11:00:00|
|1325#0x0,0x1|    48,27|2021-01-06 12:00:00|
|1325#0x0,0x1|    49,24|2021-01-06 13:00:00|
|1325#0x0,0x1|    49,25|2021-01-06 14:00:00|
|1325#0x0,0x1|    49,22|2021-01-06 15:00:00|
|1325#0x0,0x1|    48,26|2021-01-06 16:00:00|
|1325#0x0,0x1|    48,27|2021-01-06 17:00:00|
|1325#0x0,0x1|    49,27|2021-01-06 18:00:00|
|1325#0x0,

In [54]:
df4.schema

StructType(List(StructField(key_str,StringType,true),StructField(value_str,StringType,true),StructField(timestamp,TimestampType,true)))

これをさらに、1分毎のデータに水増しする。ただし、1分単位で電源をOn/Offしたり、設定をいじることはないので、ここでは簡単化のために1時間は同じ設定を維持することにする。（つまり、単にタイムスタンプだけいじって行を水増しする）

In [55]:
@udf(ArrayType(TimestampType()))
def transform_timestamp(timestamp):
    records = [timestamp]
    num_records = 60
    for i in range(0, num_records):
        delta = dt.timedelta(minutes=i+1)
        records.append(timestamp + delta)
        
    return records

In [56]:
df5 = df4.select(explode(transform_timestamp(df4['timestamp'])).alias('timestamp_min'), '*')

In [57]:
df5.show()

+-------------------+------------+---------+-------------------+
|      timestamp_min|     key_str|value_str|          timestamp|
+-------------------+------------+---------+-------------------+
|2021-01-06 00:00:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:01:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:02:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:03:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:04:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:05:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:06:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:07:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:08:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:09:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:10:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:11:00|1325#0x0,0x1|    49,22|2021-01-06 00:00:00|
|2021-01-06 00:12:00|1325

出力先に書き込み

In [58]:
df6 = df5.select(df5['timestamp_min'].alias('timestamp'), df5['key_str'].alias('id'), df5['value_str'].alias('state'))

In [59]:
df7 = df6.orderBy(df6['timestamp'], df6['id']).coalesce(1)

In [60]:
df7.write.format('csv').option('sep', r'/t').option('header', 'true').mode('overwrite').save(output_url)

書き込まれたことを確認する。

In [61]:
df = spark.read.format('csv').option('sep', r'/t').option('header', 'true'). load(output_url)
df.show()

+--------------------+------------+-----+
|           timestamp|          id|state|
+--------------------+------------+-----+
|2021-01-06T00:00:...|1003#0x0,0x1|48,29|
|2021-01-06T00:00:...|1039#0x0,0x1|48,27|
|2021-01-06T00:00:...|1061#0x0,0x1|48,29|
|2021-01-06T00:00:...|1076#0x0,0x1|48,26|
|2021-01-06T00:00:...|1113#0x0,0x1|49,26|
|2021-01-06T00:00:...|1141#0x0,0x1|48,25|
|2021-01-06T00:00:...|1195#0x0,0x1|49,26|
|2021-01-06T00:00:...|1206#0x0,0x1|49,27|
|2021-01-06T00:00:...|1217#0x0,0x1|48,24|
|2021-01-06T00:00:...|1219#0x0,0x1|49,23|
|2021-01-06T00:00:...|1257#0x0,0x1|48,25|
|2021-01-06T00:00:...|1258#0x0,0x1|49,29|
|2021-01-06T00:00:...|1268#0x0,0x1|48,28|
|2021-01-06T00:00:...|1325#0x0,0x1|49,28|
|2021-01-06T00:00:...|1375#0x0,0x1|49,28|
|2021-01-06T00:00:...|1390#0x0,0x1|49,29|
|2021-01-06T00:00:...|1394#0x0,0x1|48,26|
|2021-01-06T00:00:...|1414#0x0,0x1|48,24|
|2021-01-06T00:00:...|1449#0x0,0x1|48,23|
|2021-01-06T00:00:...|1451#0x0,0x1|48,23|
+--------------------+------------